In [9]:
import torch
import pandas as pd
import json
import os
import sys

scripts_dir = os.getcwd()
# Go up one level
project_root = os.path.abspath(os.path.join(scripts_dir, '..'))
sys.path.append(project_root)

from src.CNN import BeamformingCNN

In [ ]:
# Load the list of records
with open("data.json", "r") as f:
    records_in = json.load(f)

# Build a DataFrame
df = pd.DataFrame(records_in)

# Helper to turn JSON dicts back into torch.cfloat tensors
def deserialize_complex_list(lst):
    return [
        torch.tensor(entry["real"]) + 1j * torch.tensor(entry["imag"])
        for entry in lst
    ]

# Apply to both columns
df["H"] = df["H"].apply(deserialize_complex_list)
df["V"] = df["V"].apply(deserialize_complex_list)
df["U"] = df["U"].apply(deserialize_complex_list)
df["W"] = df["W"].apply(deserialize_complex_list)

In [ ]:
def train_val_split_indices(df, train_ratio=0.8, random_seed=None):
    n = len(df) 
    indices = np.arange(n)
    
    if random_seed is not None:
        np.random.seed(random_seed)
    np.random.shuffle(indices)
    
    # how many go to train
    n_train = int(np.floor(train_ratio * n))
    
    train_idx = indices[:n_train]
    val_idx   = indices[n_train:]
    return train_idx, val_idx

train_idx, val_idx = train_val_split_indices(df, train_ratio=0.8, random_seed=None)


In [ ]:
df_train = df.iloc[indices]
df_val = df.iloc[indices]
df

In [ ]:
model = CNN()
model.train(H_train, V_train,)


In [15]:
def generate_random_mimo_channels(n_rx, n_tx, dtype=torch.complex64):
    real_part = torch.randn(n_rx, n_tx)
    imag_part = torch.randn(n_rx, n_tx)
    channels = torch.complex(real_part, imag_part).to(dtype)
    return channels

H_l = []
for _ in range(100):
    H = {}
    for k in range(3):
        H[k] = generate_random_mimo_channels(n_rx = 2, n_tx = 4, dtype=torch.complex64)
    H_l.append(H)

In [21]:
H_stack = torch.stack([H[0], H[1], H[2]])

In [22]:
H_real = H_stack.real                                             # [K, Nr, Nt]
H_imag = H_stack.imag                                             # [K, Nr, Nt]
X = torch.cat([H_real, H_imag], dim=0) 


In [23]:
X.shape

torch.Size([6, 2, 4])

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define the CNN model
class ChannelCNN(nn.Module):
    def __init__(self, K, Nr, Nt):
        super(ChannelCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=2*K, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * Nr * Nt, 128)
        self.fc2 = nn.Linear(128, 2*K*Nr*Nt)  # Output size matches the beamforming matrix

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        x = x.view(x.size(0), 2*K, Nr, Nt)  # Reshape to match the target beamforming matrix
        return x

    def predict(self, x):
        self.eval()
        with torch.no_grad():
            return self.forward(x)

# Generate some example data
batch_size = 32
K = 4  # Example value for K
Nr = 8  # Example value for Nr
Nt = 8  # Example value for Nt
num_samples = 1000

# Randomly generated example data and beamforming matrices
data = torch.randn(num_samples, 2*K, Nr, Nt)
beamforming_matrices = torch.randn(num_samples, 2*K, Nr, Nt)

# Create DataLoader
dataset = TensorDataset(data, beamforming_matrices)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize the model, loss function, and optimizer
model = ChannelCNN(K, Nr, Nt)
criterion = nn.MSELoss()  # Using Mean Squared Error loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

print("Training complete!")

# Example prediction
example_input = torch.randn(batch_size, 2*K, Nr, Nt)
predicted_output = model.predict(example_input)
print(predicted_output.shape)  # Should be (batch_size, 2*K, Nr, Nt)


Epoch 1/10, Loss: 1.0174692869186401
Epoch 2/10, Loss: 0.9701234698295593
Epoch 3/10, Loss: 1.006143569946289
Epoch 4/10, Loss: 0.9876295328140259
Epoch 5/10, Loss: 0.9602190256118774
Epoch 6/10, Loss: 0.9687371253967285
Epoch 7/10, Loss: 0.9478237628936768
Epoch 8/10, Loss: 0.8803983926773071
Epoch 9/10, Loss: 0.9592944383621216
Epoch 10/10, Loss: 0.8480824828147888
Training complete!
torch.Size([32, 8, 8, 8])
